# Save and Load TensorFlow/Keras models

Greatly expanded from the original documentation by Umberto Michelucci.

Original code copyright 2019 The TensorFlow Authors.

# Save and load models

Model progress can be saved during and after training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

Caution: TensorFlow models are code and it is important to be careful with untrusted code. See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) for details.

### Options

There are different ways to save TensorFlow models depending on the API you're using. This guide uses [tf.keras](https://www.tensorflow.org/guide/keras)—a high-level API to build and train models in TensorFlow. The new, high-level `.keras` format used in this tutorial is recommended for saving Keras objects, as it provides robust, efficient name-based saving that is often easier to debug than low-level or legacy formats. For more advanced saving or serialization workflows, especially those involving custom objects, please refer to the [Save and load Keras models guide](https://www.tensorflow.org/guide/keras/save_and_serialize). For other approaches, refer to the [Using the SavedModel format guide](../../guide/saved_model.ipynb).


## Saving and Loading Models in Keras

Certainly! In Keras, a popular deep learning framework, you can save and load neural network models using the `save` and `load_model` functions. These functionalities are essential for preserving your trained models for future use or sharing them with others.

### Saving a Keras Model:
You can save a Keras model, including its architecture, weights, optimizer state, and any additional configuration, to a file using the `save` method. Typically, you'll save your model in either the Hierarchical Data Format (HDF5) or the TensorFlow SavedModel format.

Here's how to save a Keras model to an HDF5 file:

```python
from keras.models import Sequential
from keras.layers import Dense

# Create a simple Sequential model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

# Compile and train the model

# Save the model to an HDF5 file
model.save('my_model.h5')
```

The `model.save('my_model.h5')` line will save the model and its weights to a file named "my_model.h5" in the current directory.

### Loading a Saved Keras Model:
To load a saved Keras model, you can use the `load_model` function. You need to provide the path to the saved model file, and it will return the loaded model.

```python
from keras.models import load_model

# Load the saved model from the HDF5 file
loaded_model = load_model('my_model.h5')

# Now, you can use the loaded_model for inference or further training
```

Make sure to compile the loaded model with the same optimizer and loss function as before, if you plan to continue training it.

Additionally, you can save and load custom objects like custom layers or loss functions by providing a `custom_objects` argument when loading the model. This allows you to recreate models that include custom components correctly.

```python
from keras.models import load_model

# Load the model with custom objects
loaded_model = load_model('my_model.h5', custom_objects={'custom_layer': custom_layer})
```

That's how you save and load Keras models, allowing you to reuse your trained models for various tasks without retraining them from scratch.

## Setup

### Installs and imports

Install and import TensorFlow and dependencies:

In [1]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.15.0


### The MNIST dataset


The MNIST dataset is a widely used benchmark dataset in the field of machine learning and computer vision. It stands for "Modified National Institute of Standards and Technology" and is a collection of handwritten digits. Here's a brief description of the MNIST dataset:

1. **Image Format:** MNIST consists of a large set of 28x28 pixel grayscale images. Each image is a black-and-white representation of a handwritten digit (0 through 9).

2. **Dataset Size:** The MNIST dataset contains a total of 70,000 images. This dataset is typically divided into two main subsets: a training set with 60,000 images and a testing set with 10,000 images.

3. **Labeling:** Each image in the dataset is associated with a corresponding label, indicating the digit it represents. The labels range from 0 to 9, matching the handwritten digits in the images.

4. **Usage:** MNIST is often used for various machine learning tasks, including digit recognition, image classification, and deep learning model training. It serves as a benchmark for evaluating the performance of different machine learning algorithms and neural networks.

5. **Popularity:** Due to its simplicity and standardized format, MNIST has been widely used as a starting point for researchers and developers to experiment with and validate new machine learning techniques. It has played a crucial role in advancing the field of deep learning.

6. **Challenges:** While MNIST is relatively straightforward, it's still a challenging dataset for tasks like digit recognition because of variations in writing styles and the presence of noise in the images.

7. **Availability:** The MNIST dataset is readily available and can be accessed through various machine learning libraries, including TensorFlow and PyTorch. Many tutorials and educational resources use MNIST as an introductory dataset for teaching and learning machine learning concepts.


To demonstrate how to save and load weights, you'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). To speed up these runs, use the first 1000 examples:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

Start by building a simple sequential model:

In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Save checkpoints during training

## ```callbacks``` in Keras

In Keras, callbacks are objects that allow you to customize and extend the behavior of your neural network model during training. Callbacks are called at different points during the training process, such as at the end of each epoch or after a batch of data is processed. They provide a way to monitor the training, make decisions based on training metrics, and even perform actions like saving the model or stopping training early.

Here are some common use cases and functionalities of callbacks in Keras:

1. **Model Checkpointing:** You can use callbacks to save the model's weights and architecture at specific intervals during training. This is useful to ensure that you can restore the best model if training is interrupted or to save multiple checkpoints for later comparison.

2. **Early Stopping:** Callbacks can monitor a specified metric (e.g., validation loss or accuracy) and stop training early if the metric doesn't improve or starts deteriorating. This prevents overfitting and saves training time.

3. **Learning Rate Scheduling:** Adjusting the learning rate during training can be crucial for better convergence. Callbacks like LearningRateScheduler allow you to change the learning rate according to a predefined schedule.

4. **Custom Logging:** You can create custom callbacks to log training metrics, custom loss functions, or any other information you want to track during training. This is helpful for creating detailed training logs or visualizations.

5. **TensorBoard Integration:** Keras provides a callback, `TensorBoard`, for integrating with TensorFlow's TensorBoard tool. TensorBoard allows you to visualize metrics, model architecture, and more during training.

6. **Data Augmentation:** Callbacks can apply data augmentation techniques during training to artificially increase the diversity of training data and improve model generalization.

7. **Custom Actions:** You can implement custom actions to be taken during training, such as sending notifications, saving intermediate results, or modifying the training process based on specific conditions.

Here's a basic example of using some common callbacks in Keras:

```python
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

# Define callbacks
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
tensorboard = TensorBoard(log_dir='logs')

# Train the model with callbacks
history = model.fit(
    x_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[model_checkpoint, early_stopping, tensorboard]
)
```

In this example, we use three common callbacks: `ModelCheckpoint` to save the best model, `EarlyStopping` to stop training if validation loss stops improving, and `TensorBoard` to log training progress for visualization.


## Checkpoint callback

You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The `tf.keras.callbacks.ModelCheckpoint` callback allows you to continually save the model both *during* and at *the end* of training.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback that saves weights only during training:

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
 1/32 [..............................] - ETA: 4s - loss: 2.1726 - sparse_categorical_accuracy: 0.1875
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 1.1159 - sparse_categorical_accuracy: 0.6960 - val_loss: 0.7410 - val_sparse_categorical_accuracy: 0.7550
Epoch 2/10
 1/32 [..............................] - ETA: 0s - loss: 0.4825 - sparse_categorical_accuracy: 0.8438
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.4224 - sparse_categorical_accuracy: 0.8710 - val_loss: 0.5102 - val_sparse_categorical_accuracy: 0.8450
Epoch 3/10
 1/32 [..............................] - ETA: 0s - loss: 0.1636 - sparse_categorical_accuracy: 1.0000
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 3ms/step - loss: 0.2780 - sparse_categorical_accuracy: 0.9270 - val_loss: 0.4645 - val_sparse_categorical_accuracy: 0.8620
Epoch 4/10
30/32 [==========

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights. 

Now rebuild a fresh, untrained model and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3403 - sparse_categorical_accuracy: 0.0980 - 76ms/epoch - 2ms/step
Untrained model, accuracy:  9.80%


Then load the weights from the checkpoint and re-evaluate:

In [8]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4316 - sparse_categorical_accuracy: 0.8600 - 43ms/epoch - 1ms/step
Restored model, accuracy: 86.00%


### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [9]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


Now, review the resulting checkpoints and choose the latest one:

In [10]:
os.listdir(checkpoint_dir)

['cp-0005.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index']

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

Note: The default TensorFlow format only saves the 5 most recent checkpoints.

To test, reset the model, and load the latest checkpoint:

In [12]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4983 - sparse_categorical_accuracy: 0.8640 - 104ms/epoch - 3ms/step
Restored model, accuracy: 86.40%


## What are these files?

The above code stores the weights to a collection of [checkpoint](../../guide/checkpoint.ipynb)-formatted files that contain only the trained weights in a binary format. Checkpoints contain:
* One or more shards that contain your model's weights.
* An index file that indicates which weights are stored in which shard.

If you are training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

## Manually save weights

To save weights manually, use `tf.keras.Model.save_weights`. By default, `tf.keras`—and the `Model.save_weights` method in particular—uses the TensorFlow [Checkpoint](../../guide/checkpoint.ipynb) format with a `.ckpt` extension. To save in the HDF5 format with a `.h5` extension, refer to the [Save and load models](https://www.tensorflow.org/guide/keras/save_and_serialize) guide.

In [13]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4983 - sparse_categorical_accuracy: 0.8640 - 71ms/epoch - 2ms/step
Restored model, accuracy: 86.40%


## Save the entire model

Call `tf.keras.Model.save` to save a model's architecture, weights, and training configuration in a single `model.keras` zip archive.

An entire model can be saved in three different file formats (the new `.keras` format and two legacy formats: `SavedModel`, and `HDF5`). Saving a model as `path/to/model.keras` automatically saves in the latest format.

**Note:** For Keras objects it's recommended to use the new high-level `.keras` format for richer, name-based saving and reloading, which is easier to debug. The low-level SavedModel format and legacy H5 format continue to be supported for existing code.

You can switch to the SavedModel format by:

- Passing `save_format='tf'` to `save()`
- Passing a filename without an extension

You can switch to the H5 format by:
- Passing `save_format='h5'` to `save()`
- Passing a filename that ends in `.h5`

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/models/convert/#convert_a_savedmodel_recommended_), [HDF5](https://www.tensorflow.org/lite/models/convert/#convert_a_keras_model_))

\*Custom objects (for example, subclassed models or layers) require special attention when saving and loading. Refer to the **Saving custom objects** section below.

### New high-level `.keras` format

The new Keras v3 saving format, marked by the `.keras` extension, is a more
simple, efficient format that implements name-based saving, ensuring what you load is exactly what you saved, from Python's perspective. This makes debugging much easier, and it is the recommended format for Keras.

The section below illustrates how to save and restore the model in the `.keras` format.

In [14]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1820 - sparse_categorical_accuracy: 0.6760
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4225 - sparse_categorical_accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.3102 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2112 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1605 - sparse_categorical_accuracy: 0.9650


Reload a fresh Keras model from the `.keras` zip archive:

In [15]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Try running evaluate and predict with the loaded model:

In [16]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4132 - sparse_categorical_accuracy: 0.8640 - 93ms/epoch - 3ms/step
Restored model, accuracy: 86.40%
32/32 [==============================] - 0s 636us/step
(1000, 10)


### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. The [SavedModel guide](../../guide/saved_model.ipynb) goes into detail about how to `serve/inspect` the SavedModel. The section below illustrates the steps to save and restore the model.

In [17]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1720 - sparse_categorical_accuracy: 0.6670
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4156 - sparse_categorical_accuracy: 0.8900
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2911 - sparse_categorical_accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2160 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1551 - sparse_categorical_accuracy: 0.9680
INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory:

In [18]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model


assets            keras_metadata.pb variables
fingerprint.pb    saved_model.pb


Reload a fresh Keras model from the saved model:

In [19]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [20]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4641 - sparse_categorical_accuracy: 0.8520 - 75ms/epoch - 2ms/step
Restored model, accuracy: 85.20%
32/32 [==============================] - 0s 604us/step
(1000, 10)


### HDF5 format

Keras provides a basic legacy high-level save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. 

In [21]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1485 - sparse_categorical_accuracy: 0.6650
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4393 - sparse_categorical_accuracy: 0.8740
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2951 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2089 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1687 - sparse_categorical_accuracy: 0.9570


/Users/umberto/miniforge3/envs/tf-jan24/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Now, recreate the model from that file:

In [22]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Check its accuracy:

In [23]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4286 - sparse_categorical_accuracy: 0.8560 - 79ms/epoch - 2ms/step
Restored model, accuracy: 85.60%


Keras saves models by inspecting their architectures. This technique saves everything:

* The weight values
* The model's architecture
* The model's training configuration (what you pass to the `.compile()` method)
* The optimizer and its state, if any (this enables you to restart training where you left off)

Keras is not able to save the `v1.x` optimizers (from `tf.compat.v1.train`) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.


### Saving custom objects

If you are using the SavedModel format, you can skip this section. The key difference between high-level `.keras`/HDF5 formats and the low-level SavedModel format is that the `.keras`/HDF5 formats uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the original code. However, debugging low-level SavedModels can be more difficult as a result, and we recommend using the high-level `.keras` format instead due to its name-based, Keras-native nature.

To save custom objects to `.keras` and HDF5, you must do the following:

1. Define a `get_config` method in your object, and optionally a `from_config` classmethod.
  * `get_config(self)` returns a JSON-serializable dictionary of parameters needed to recreate the object.
  * `from_config(cls, config)` uses the returned config from `get_config` to create a new object. By default, this function will use the config as initialization kwargs (`return cls(**config)`).
2. Pass the custom objects to the model in one of three ways:
  - Register the custom object with the `@tf.keras.utils.register_keras_serializable` decorator. **(recommended)**
  - Directly pass the object to the `custom_objects` argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`
  - Use a `tf.keras.utils.custom_object_scope` with the object included in the `custom_objects` dictionary argument, and place a `tf.keras.models.load_model(path)` call within the scope.

Refer to the [Writing layers and models from scratch](https://www.tensorflow.org/guide/keras/custom_layers_and_models) tutorial for examples of custom objects and `get_config`.


# APPENDIX A - The HDF5 Format

HDF5 (Hierarchical Data Format version 5) is a widely used file format for storing and managing large and complex datasets or models. It was designed to address the needs of scientific and engineering applications that require efficient and flexible data storage and retrieval. HDF5 is platform-independent and supports hierarchical data structures, making it an excellent choice for storing a wide range of data types, including numerical data, images, text, and more. Here are some key characteristics and features of the HDF5 format:

1. **Hierarchical Structure:** HDF5 files are organized into a hierarchical structure, similar to a file system. They consist of groups and datasets, where groups can contain other groups or datasets. This hierarchical organization allows for logical grouping and management of data.

2. **Support for Multiple Data Types:** HDF5 supports various data types, including integers, floating-point numbers, strings, compound data types, and more. This flexibility makes it suitable for storing diverse data types within the same file.

3. **Chunking:** HDF5 allows data to be stored in chunks, which can improve data access and compression efficiency. Users can define the chunk size based on their specific data access patterns.

4. **Compression:** HDF5 provides built-in support for data compression, which can significantly reduce file size while maintaining data integrity. You can choose from several compression algorithms, such as GZIP or LZF, depending on your needs.

5. **Metadata:** HDF5 files can store metadata, which includes attributes and annotations that describe the data within the file. This metadata can be used to document and provide context for the stored data.

6. **Data Slicing:** HDF5 allows for efficient data slicing, enabling users to access and manipulate specific portions of large datasets without loading the entire dataset into memory.

7. **Cross-Platform Compatibility:** HDF5 files are designed to be platform-independent, meaning they can be created on one platform and read on another without compatibility issues. This makes it suitable for sharing data across different operating systems and architectures.

8. **Language Support:** HDF5 libraries are available for various programming languages, including Python, C/C++, MATLAB, Java, and more. This wide range of language support makes it accessible to a broad user base.

9. **Parallel I/O:** HDF5 provides support for parallel I/O, which is essential for high-performance computing applications that involve multiple processors or nodes.

10. **Extensibility:** HDF5 allows users to define their own data structures and data types, making it versatile for handling custom data formats and requirements.

## HDF5 Format for Keras Models

The HDF5 (Hierarchical Data Format version 5) format is commonly used to store Keras models because it provides a structured way to save all the necessary information about a neural network model. When you save a Keras model in HDF5 format, you're essentially saving the model's architecture, weights, optimizer configuration, and any custom objects it may contain. Here's an overview of how Keras models are stored in HDF5 format:

1. **Model Architecture:** The HDF5 file stores the complete model architecture, including all the layers and their configurations. This includes information about layer types, activation functions, input shapes, and any other relevant layer-specific settings. This information is saved in a structured manner, making it possible to reconstruct the model's architecture accurately when you load it later.

2. **Model Weights:** The HDF5 file contains the model's weight parameters. These weights represent the learned parameters of the model, such as the weights and biases of each layer's neurons. Saving and loading weights is essential for preserving the knowledge gained during training.

3. **Optimizer State:** If you save a model after training it, the HDF5 file includes the state of the optimizer. This includes the optimizer's configuration and the current values of all its internal variables. When you load the model, it restores the optimizer state, allowing you to continue training from where you left off.

4. **Custom Objects:** If your Keras model includes custom layers, loss functions, or other custom objects, the HDF5 format allows you to save and load these custom components. However, you may need to provide a `custom_objects` dictionary when loading the model to tell Keras how to reconstruct these custom objects correctly.

Here's a basic example of how to save a Keras model in HDF5 format:

```python
from keras.models import Sequential
from keras.layers import Dense

# Create a simple Sequential model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

# Compile and train the model

# Save the model to an HDF5 file
model.save('my_model.h5')
```

To load the model later:

```python
from keras.models import load_model

# Load the saved model from the HDF5 file
loaded_model = load_model('my_model.h5')

# Now, you can use the loaded_model for inference or further training
```
